In [1]:
import numpy as np
class EcoSystem():
    # EcoSystem - vanilla GA algorithm for feature selection in Hackerrank's Dota challange
    def __init__(self, 
                 ecosystem_size, 
                 population_size, 
                 chromosome_sizes,
                 chromosome_bases,
                 evaluator,
                 mutation_coef = [0.01],
                 elitism_coef = 0.05):
        
        # Input tests
        assert(population_size % 2 == 0)
        
        # Creating EcoSystem
        self.ECOSYSTEM_SIZE = ecosystem_size
        self.POPULATION_SIZE = population_size
        self.CHROMOSOME_SIZES = chromosome_sizes
        self.CHROMOSOME_BASES = chromosome_bases
        self.EVALUATOR = evaluator
        self.MUTATION_COEF = mutation_coef
        self.ELITISM_COEF = elitism_coef
        self.ecosystem = []
        
        # Imbueing EcoSystem with life        
        for i in xrange(self.ECOSYSTEM_SIZE):
            self.ecosystem.append(
                Population(
                    self.POPULATION_SIZE,
                    self.CHROMOSOME_SIZES,
                    self.CHROMOSOME_BASES,
                    self.EVALUATOR,
                    self.MUTATION_COEF,
                    self.ELITISM_COEF                   
                )
            )            
            self.ecosystem[-1].grow_populations()

    def evolve(self, epochs):        
        self.EPOCHS = epochs
        
        self.population_solutions = []
        
        for population in self.ecosystem:
            for epoch in xrange(self.EPOCHS):
                
                self.ecosystem[-1].evaluate_population()
                self.ecosystem[-1].select()
                self.ecosystem[-1].crossover()
                self.ecosystem[-1].mutation()
                self.ecosystem[-1].elite_preservation()
                
            self.ecosystem[-1].evaluate_population()
            self.population_solutions.append([population.fittest_genome_score, population.fittest_genome])
            
    
class Population():
    
    def __init__(self, population_size, chromosome_sizes, chromosome_bases, evaluator, mutation_coef, elitism_coef):
        # Creating populations
        self.POPULATION_SIZE = population_size
        self.CHROMOSOME_SIZES = chromosome_sizes
        self.CHROMOSOME_BASES = chromosome_bases
        self.EVALUATOR = evaluator
        self.MUTATION_COEF = mutation_coef        
        self.ELITES_NUM = int(self.POPULATION_SIZE * elitism_coef)
        
        self.scores = np.zeros(np.sum(self.CHROMOSOME_SIZES))
        
        self.genome = []
        self.elite_genome = []
        self.fittest_genome = [None] * len(self.CHROMOSOME_SIZES)
        
    def grow_populations(self):
        # Initializing population with random binary data        
        for i in xrange(len(self.CHROMOSOME_SIZES)):
            self.genome.append(
                np.random.randint(
                    0,
                    self.CHROMOSOME_BASES[i],
                    size = (self.POPULATION_SIZE, self.CHROMOSOME_SIZES[i])
                )
            )
            self.elite_genome.append(np.empty((self.ELITES_NUM, self.CHROMOSOME_SIZES[i])))
            
    def evaluate_population(self):
        # Evaluating population
        self.scores = self.EVALUATOR(self.genome)
        
        fittest_genome_index = np.argmin(self.scores)
        self.fittest_genome_score = self.scores[fittest_genome_index]
        
        for i in xrange(len(self.genome)):
            self.fittest_genome[i] = self.genome[i][fittest_genome_index, :]
            
        elite_indices = np.argpartition(self.scores, self.ELITES_NUM)[: self.ELITES_NUM]
        
        for i in xrange(len(self.genome)):
            self.elite_genome[i] = np.take(self.genome[i], elite_indices, axis = 0)
        
    def select(self):
        # Tournament participant pairs selection indices
        tournament_selector = np.random.randint(0, self.POPULATION_SIZE, (2, self.POPULATION_SIZE))
        
        # Tournament
        winner_selector_indices = np.argmin(np.take(self.scores, tournament_selector), axis = 0)
        winner_indices = tournament_selector[0]
        winner_indices[winner_selector_indices == 1] = tournament_selector[1][winner_selector_indices == 1]
        
        for chromosome in self.genome:            
            chromosome[:, :] = np.take(chromosome, winner_indices, axis = 0)
            
    def crossover(self):
        # Uniform crossover operator
        for chromosome in self.genome:
            crossover_selector = np.random.randint(0, 2, (chromosome.shape[0] / 2, chromosome.shape[1]))
            for i in xrange(crossover_selector.shape[0]):                
                tmp = chromosome[i * 2, crossover_selector[i, :] == 1]
                chromosome[i * 2, crossover_selector[i, :] == 1] = chromosome[i * 2 + 1, crossover_selector[i, :] == 1]
                chromosome[i * 2 + 1, crossover_selector[i, :] == 1] = tmp
            
    def mutation(self):
        # Introducing uniform mutation
        for i in xrange(len(self.genome)):
            mutation_selector = np.random.binomial(1, self.MUTATION_COEF[i], self.genome[i].shape)
            self.genome[i][mutation_selector == 1] = np.random.randint(
                0,
                self.CHROMOSOME_BASES[i],
                np.sum(mutation_selector == 1)
            )

    def elite_preservation(self):
        # Elites taking their rightful place
        for i in xrange(len(self.genome)):
            self.genome[i][: self.ELITES_NUM, :] = self.elite_genome[i]